In [ ]:
from transformers import pipeline
import torch
import torch.nn as nn

 ## Training a new tokenizer

### Import Dataset and create train, test, validation corpus from it

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Raw code dataset loaded in to raw_datasets dictionary and aplit in to train, validation and test daasets
raw_datasets = load_dataset("code_search_net", "python")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [ ]:
print(raw_datasets["train"], type(raw_datasets))

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
}) <class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
print(raw_datasets["train"][123]["func_code_tokens"])

['def', 'ne', '(', 'name', ',', 'value', ')', ':', 'ret', '=', '{', "'name'", ':', 'name', ',', "'result'", ':', 'False', ',', "'comment'", ':', "''", ',', "'changes'", ':', '{', '}', '}', 'if', 'name', 'not', 'in', '__reg__', ':', 'ret', '[', "'result'", ']', '=', 'False', 'ret', '[', "'comment'", ']', '=', "'Value {0} not in register'", '.', 'format', '(', 'name', ')', 'return', 'ret', 'if', '__reg__', '[', 'name', ']', '[', "'val'", ']', '!=', 'value', ':', 'ret', '[', "'result'", ']', '=', 'True', 'return', 'ret']


In [ ]:
# transform the dataset into batches of 1000 which can be loaded in memory as needed. Using  Generator function
#will enable our tokenizer to go faster (training on batches of texts instead of processing individual texts one by one)
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

training_corpus = get_training_corpus()


In [ ]:
training_corpus

<generator object get_training_corpus at 0x79a7658273e0>

### Training a new tokenizer

In [ ]:
# We are importing architecture of gpt2 tokenizer and training it with our dataset
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Example with old tokenizer
example = '''def add_numbers(a, b):"""Add the two numbers `a` and `b`."""return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 '"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'return',
 'Ġa',
 'Ġ+',
 'Ġb']

In [ ]:
# Lets train new tokenizer
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 '"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'return',
 'Ġa',
 'Ġ+',
 'Ġb']

In [ ]:
tokenizer.save_pretrained("code-search-net-tokenizer")

('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.json',
 'code-search-net-tokenizer/merges.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')

### Saving the tokenizer to Hugginface hub account for future reuse

In [ ]:
tokenizer.push_to_hub("code-search-net-tokenizer")

ValueError: ignored

In [ ]:
tokenizer_new = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

In [ ]:
encoding = tokenizer(example)

In [ ]:
print(encoding.word_ids(),'\n', encoding.tokens())

## Byte Pair Encoding (BPE) Tokenizer

In [21]:
from collections import defaultdict
import re
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

word_freqs = defaultdict(int)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
# Example Corpus used for learning tokens for BLE
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [4]:
# Pre-tokenization - converting sentences to words as pre-tokens and then convert the words to a list of characters
word_freq_dict = {}
character_list = []
temp_set = set()
character_set = set()
for text in corpus:
  pattern = r'\w+|[.]|[,]'

  # Find all matches of the pattern in the sentence
  words = re.findall(pattern, text)


  for word in words:
    word_freq_dict[word] = word_freq_dict.get(word, 0) + 1
    temp_set = set(word)
    character_set = character_set | temp_set
    character_list = list(character_set)
    character_list.sort()

print(word_freq_dict, '\n', character_list)

{'This': 3, 'is': 2, 'the': 1, 'Hugging': 1, 'Face': 1, 'Course': 1, '.': 4, 'chapter': 1, 'about': 1, 'tokenization': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms': 1, 'Hopefully': 1, ',': 1, 'you': 1, 'will': 1, 'be': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens': 1} 
 [',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']


In [5]:
vocab = ["<|endoftext|>"] + character_list.copy()
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']


In [6]:
splits = {word: [c for c in word] for word in word_freq_dict.keys()}
print(splits)

{'This': ['T', 'h', 'i', 's'], 'is': ['i', 's'], 'the': ['t', 'h', 'e'], 'Hugging': ['H', 'u', 'g', 'g', 'i', 'n', 'g'], 'Face': ['F', 'a', 'c', 'e'], 'Course': ['C', 'o', 'u', 'r', 's', 'e'], '.': ['.'], 'chapter': ['c', 'h', 'a', 'p', 't', 'e', 'r'], 'about': ['a', 'b', 'o', 'u', 't'], 'tokenization': ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n'], 'section': ['s', 'e', 'c', 't', 'i', 'o', 'n'], 'shows': ['s', 'h', 'o', 'w', 's'], 'several': ['s', 'e', 'v', 'e', 'r', 'a', 'l'], 'tokenizer': ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'], 'algorithms': ['a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'], 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'], ',': [','], 'you': ['y', 'o', 'u'], 'will': ['w', 'i', 'l', 'l'], 'be': ['b', 'e'], 'able': ['a', 'b', 'l', 'e'], 'to': ['t', 'o'], 'understand': ['u', 'n', 'd', 'e', 'r', 's', 't', 'a', 'n', 'd'], 'how': ['h', 'o', 'w'], 'they': ['t', 'h', 'e', 'y'], 'are': ['a', 'r', 'e'], 'trained': ['t', 'r', 'a', 'i', 'n', 'e

In [7]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freq_dict.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [15]:
def merge_pair(a, b, splits):
    for word in word_freq_dict:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [13]:
vocab_size = 50
merges = {}

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [14]:
print(merges)

{}


In [24]:
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'er', 'to', 'en', 'Th', 'This', 'th', 'ou', 'se', 'tok', 'token', 'nd', 'the', 'in', 'ab', 'tokeni', 'tokeniz', 'at', 'io', 'ion', 'ho']


In [22]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [23]:
tokenize("This is not a token.")

['T',
 'h',
 'i',
 's',
 'Ġ',
 'i',
 's',
 'Ġ',
 'n',
 'o',
 't',
 'Ġ',
 'a',
 'Ġ',
 't',
 'o',
 'k',
 'e',
 'n',
 '.']

## WordPiece Tokenizer

In [25]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [36]:
# Pre-tokenization - converting sentences to words as pre-tokens and then convert the words to a list of characters
word_freq_dict = {}
character_list = []
temp_set = set()
character_set = set()
for text in corpus:
  pattern = r'\w+|[.]|[,]'

  # Find all matches of the pattern in the sentence
  words = re.findall(pattern, text)


  for word in words:
    word_freq_dict[word] = word_freq_dict.get(word, 0) + 1
    temp_set = set(word)
    character_set = character_set | temp_set
    character_list = list(character_set)


for word in word_freq_dict.keys():
  for letter in word[1:]:
      if f"##{letter}" not in character_list:
        character_list.append(f"##{letter}")
  character_list.sort()

print(word_freq_dict, '\n', character_list)

{'This': 3, 'is': 2, 'the': 1, 'Hugging': 1, 'Face': 1, 'Course': 1, '.': 4, 'chapter': 1, 'about': 1, 'tokenization': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms': 1, 'Hopefully': 1, ',': 1, 'you': 1, 'will': 1, 'be': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens': 1} 
 ['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']


In [37]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + character_list.copy()

In [39]:
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freq_dict.keys()
}

In [41]:
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freq_dict.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [42]:
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('T', '##h'): 0.125
('##h', '##i'): 0.03409090909090909
('##i', '##s'): 0.02727272727272727
('i', '##s'): 0.1
('t', '##h'): 0.03571428571428571
('##h', '##e'): 0.011904761904761904


In [44]:
def merge_pair(a, b, splits):
    for word in word_freq_dict:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [45]:
splits = merge_pair("a", "##b", splits)
splits["about"]

['ab', '##o', '##u', '##t']

In [46]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [51]:
print(vocab)


['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', '##fu', 'Fa', 'Fac', '##ct', '##ful', '##full', '##fully', 'Th', 'ch', '##hm', 'cha', 'chap', 'chapt']


## Creating Tokenizer from scratch

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

# Generator function to get batches of 1000 datasets
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [7]:
tokenizer.normalizer = normalizers.Sequence([normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()])

In [9]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [10]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [11]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [13]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens, '\n', encoding.ids,'\n', encoding.type_ids)

['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.'] 
 [2817, 11, 61, 3409, 1317, 24116, 18701, 6411, 18] 
 [0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [15]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [16]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [18]:
tokenizer.decode(encoding.ids)

"let ' s test this tokenizer."

In [17]:
tokenizer.save("tokenizer.json")

In [19]:
new_tokenizer = Tokenizer.from_file("tokenizer.json")

In [20]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)